In [ ]:
%matplotlib inline
import torch, os, torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
torch.__version__

### Fine tuning 模型微调

#### 什么是微调
针对于某个任务，自己的训练数据不多，那怎么办？ 没关系，我们先找到一个同类的别人训练好的模型，把别人现成的训练好了的模型拿过来，换成自己的数据，调整一下参数，再训练一遍，这就是微调（fine-tune）。`PyTorch`里面提供的经典的网络模型都是官方通过Imagenet的数据集与训练好的数据，如果我们的数据训练数据不够，这些数据是可以作为基础模型来使用的。

#### 为什么要微调
1. 对于数据集本身很小（几千张图片）的情况，从头开始训练具有几千万参数的大型神经网络是不现实的，因为越大的模型对数据量的要求越大，过拟合无法避免。这时候如果还想用上大型神经网络的超强特征提取能力，只能靠微调已经训练好的模型。
2. 可以降低训练成本：如果使用导出特征向量的方法进行迁移学习，后期的训练成本非常低，用 CPU 都完全无压力，没有深度学习机器也可以做。
3. 前人花很大精力训练出来的模型在大概率上会比你自己从零开始搭的模型要强悍，没有必要重复造轮子。

#### 迁移学习 Transfer Learning
迁移学习是机器学习的分支, 大家看到的迁移学习基本上都是以神经网络相关的计算机视觉为主。

迁移学习初衷是节省人工标注样本的时间，让模型可以通过一个已有的标记数据的领域向未标记数据领域进行迁移从而训练出适用于该领域的模型，直接对目标域从头开始学习成本太高，我们故而转向运用已有的相关知识来辅助尽快地学习新知识。

迁移学习按照学习方式可以分为基于样本的迁移，基于特征的迁移，基于模型的迁移，以及基于关系的迁移。


#### 二者关系

其实 "Transfer Learning" 和 "Fine-tune" 并没有严格的区分，含义可以相互交换，只不过后者似乎更常用于形容迁移学习的后期微调中。 

### 如何微调

对于不同的领域微调的方法也不一样，比如语音识别领域一般微调前几层，图片识别问题微调后面几层。

对于图片来说，我们CNN的前几层学习到的都是低级的特征，比如，点、线、面，这些低级的特征对于任何图片来说都是可以抽象出来的，所以我们将他作为通用数据，只微调这些低级特征组合起来的高级特征即可，例如，这些点、线、面，组成的是圆还是椭圆，还是正方形，这些代表的含义是我们需要后面训练出来的。

对于语音来说，每个单词表达的意思都是一样的，只不过发音或者是单词的拼写不一样，比如 苹果，apple，apfel（德语），都表示的是同一个东西，只不过发音和单词不一样，但是他具体代表的含义是一样的，就是高级特征是相同的，所以我们只要微调低级的特征就可以了。

下面只介绍下计算机视觉方向的微调
- ConvNet as fixed feature extractor.： 

    其实这里有两种做法：
    1. 使用最后一个fc layer之前的fc layer获得的特征，学习个线性分类器(比如SVM)
    2. 重新训练最后一个fc layer

- Fine-tuning the ConvNet
固定前几层的参数，只对最后几层进行fine-tuning。

对于上面两种方案有一些微调的小技巧，比如先计算出预训练模型的卷积层对所有训练和测试数据的特征向量，然后抛开预训练模型，只训练自己定制的简配版全连接网络。 这个方式的一个好处就是节省计算资源，每次迭代都不会再去跑全部的数据，而只是跑一下简配的全连接。

- Pretrained models
这个其实和第二种是一个意思，不过比较极端，使用整个pre-trained的model作为初始化，然后fine-tuning整个网络而不是某些层，但是这个的计算量是非常大的,就只相当于做了一个初始化。


### 微调实例
这里面我们使用官方训练好的resnet50来参加kaggle上面的 dog breed 狗的种类识别来做一个简单微调实例。

首先我们需要下载官方的数据解压，只要保持数据的目录结构即可，这里指定一下目录的位置，并且看下内容

In [ ]:
DATA_ROOT = 'data'
all_labels_df = pd.read_csv(os.path.join(DATA_ROOT, 'labels.csv'))
all_labels_df.head()

获取狗的分类根据分类进行编号

这里定义了两个字典，分别以名字和id作为对应，方便后面处理

In [ ]:
breeds = all_labels_df['breed'].unique()
breed2idx = dict((breed, idx) for idx, breed in enumerate(breeds))
idx2breed = dict((idx, breed) for idx, breed in enumerate(breeds))
len(breeds)

添加到列表中

In [ ]:
all_labels_df['label_idx'] = [breed2idx[b] for b in all_labels_df.breed]
all_labels_df.head()

由于我们的数据集不是官方指定的格式，我们自己定义一个数据集

In [ ]:
class DogDataset(Dataset):
    def __init__(self, labels_df, img_path, transform = None):
        self.labels_df = labels_df
        self.img_path = img_path
        self.transform = transform

    def __len__(self):
        return self.labels_df.shape[0]

    def __getitem__(self, idx):
        image_name = os.path.join(self.img_path, self.labels_df.id[idx] + '.jpg')
        img = Image.open(image_name)
        label = self.labels_df.label_idx[idx]

        if self.transform:
            img = self.transform(img)

            return img, label


定义一些超参数

In [ ]:
IMG_SIZE = 224     # resnet50的输入是224的所以需要将图片统一大小
BATCH_SIZE = 256   #这个批次大小需要占用4.6-5g的显存，如果不够的化可以改下批次，如果内存超过10G可以改为512
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]
CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if CUDA else "cpu")

定义训练和验证数据的图片变换规则

In [ ]:
train_transforms= transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])

val_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])

我们这里只分割10%的数据作为训练时的验证数据

In [ ]:
dataset_names = ['train', 'valid']
stratified_split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.1, random_state = 0)
train_split_idx, valid_split_idx = next(iter(stratified_split.split(all_labels_df.id, all_labels_df.breed)))
train_df = all_labels_df.iloc[train_split_idx].reset_index()
valid_df = all_labels_df.iloc[valid_split_idx].reset_index()
print(len(train_df))
print(len(valid_df))


使用官方的dataloader载入数据

In [ ]:
img_transforms = {'train': train_transforms, 'valid': val_transforms}

train_dataset = DogDataset(train_df, os.path.join(DATA_ROOT, 'train'), transform = img_transforms['train'])
valid_dataset = DogDataset(valid_df, os.path.join(DATA_ROOT, 'train'), transform = img_transforms['valid'])
image_datasets = {'train': train_dataset, 'valid': valid_dataset}

image_dataloader = {x: DataLoader(image_datasets[x], batch_size = BATCH_SIZE, shuffle = True, num_workers = 0) for x in dataset_names}
dataset_sizes = {x: len(image_datasets[x]) for x in dataset_names}

开始配置网络，由于ImageNet是识别1000个物体，我们的狗的分类一共只有120，所以需要对模型的最后一层全连接层进行微调，将输出从1000改为120

In [ ]:
model_ft = models.resnet50(pretrained = True)

for param in model_ft.parameters():
    param.requires_grad = False

# 这里打印下全连接层的信息
print(model_ft.fc)   
num_fc_ftr = model_ft.fc.in_features  # 获取全连接层的输入特征数
model_ft.fc = nn.Linear(num_fc_ftr, len(breeds))   #  定义一个新的FC层
model_ft = model_ft.to(DEVICE)  # 将模型移动到GPU上
print(model_ft)  # 打印下网络结构

In [ ]:
# 设置训练参数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model_ft.fc.parameters()}
], lr = 0.0001)

# 定义训练函数

def train(model, device, train_loader, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} Loss: {:.6f}'.format(epoch, loss.item()))


# 定义测试函数

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            test_loss += criterion(y_hat, y).item()
            pred = y_hat.max(1, keepdim = True)[1]  # 获取最大概率的索引
            correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_dataset), 100. * correct / len(valid_dataset)))
    
    
            


训练9次，看看效果

In [ ]:
import time

for epoch in range(1, 10):
    start_time = time.time()
    train(model = model_ft, device = DEVICE, train_loader = image_dataloader['train'], epoch = epoch)
    print(f"训练用时: {time.time() - start_time:.2f}秒")
    test(model = model_ft, device = DEVICE, test_loader = image_dataloader['valid'])

In [ ]:
但是每次训练都需要将一张图片在全部网络中进行计算，而且计算的结果每次都是一样的，这样浪费了很多计算的资源。 下面我们就将这些不进行反向传播或者说不更新网络权重参数层的计算结果保存下来，这样我们以后使用的时候就可以直接将这些结果输入到FC层或者以这些结果构建新的网络层，省去了计算的时间，并且这样如果只训练全连接层，CPU就可以完成了。